# Pointcloud version including PyTorch Geometric
This notebook tests the functionality of the second implemenation of the GCN. Here we implement both a pointcloud version and a mesh version seperately on every component level. This version gets the help from PyTorch Geometric, which should speedup the application and make programming easier. The Mesh version is there to justify this version as a reimplementation of the paper and the pointcloud version is the purpose of the research.
## Structure Notebook
The following secions will be here to justify the implementation of the second verion.
- Pre-processing data
    - Create artificial noise (Store Ground Truth)
    - Patch selection
    - Patch alignment (Classify Patch)
    - Converting patch to graph
    - Storing graph in file
- Training GCN
    - Defining input and output for network
    - Create GCN architecture
        - Must be modifiable! (Create hyperparameters)
    - Selecting and loading data
        - Select based on object, noise level and patch class
        - Select based on training, validation and test group
    - Train network
        - Create log structure to view training process.
        - Be able to start and stop training.
        - Save and log training progress, such that is can be fully recreated (Per training session log: Architecture, Model weights, Hyperparameter settings, Epochs trained, Trainingset per Epoch, )
        - LOG EVERYTHING
- Post-processing data
    - Create pipeline for pre-processing data and input the network without storing data in files.
    - Reverse patch alignment of network output.
    - Create Vertex updating based on normals.
    - Create normal refinement (bilateral filter).
# Settings

In [13]:
from Pointcloud.Modules.Noise import Noise
from Pointcloud.Modules.Object import Pointcloud, Mesh, HelperFunctions
from Pointcloud.Modules.PatchGenerator import PatchGenerator
from Pointcloud.Modules.Visualize import visualize_coloring

from copy import deepcopy
from igl import barycenter as igl_barycenter, bounding_box_diagonal as igl_bounding_box_diagonal, triangle_triangle_adjacency as igl_triangle_triangle_adjacency, avg_edge_length as igl_avg_edge_length
from meshplot import plot as mp_plot
from gravomg.util import neighbors_from_stiffness
import numpy as np
from numpy import random as np_random
from pathlib import Path
import matplotlib.pyplot as plt
from robust_laplacian import point_cloud_laplacian
from scipy.spatial import Delaunay, KDTree
from sklearn.preprocessing import normalize as sklearn_preprocessing_normalize
from timeit import timeit
import torch_geometric as pyg
from torch_cluster import knn_graph
import torch

EXAMPLE_OBJ_FILE = "common-3d-test-models-master/stanford-bunny.obj"

# Pre-processing
In this section we show that all pre-processing steps are taken in the correct fashion.
## Loading (and calculating) object data
In this section we show that loading the example mesh can be done and that the object can be viewed as pointcloud and as mesh (with normals per face and normals per vertex)

In [2]:
pointcloud = Pointcloud(EXAMPLE_OBJ_FILE)
mesh = Mesh(EXAMPLE_OBJ_FILE)

In [3]:
normal_display_length = igl_bounding_box_diagonal(pointcloud.v) / 100
starts = pointcloud.v
ends = starts + pointcloud.vn * normal_display_length
pplot = mp_plot(starts, shading={"point_size": 0.005})
pplot.add_lines(starts, ends)
barycenters = igl_barycenter(mesh.v, mesh.f)
mplot = mp_plot(mesh.v, mesh.f)
_ = mplot.add_lines(barycenters, barycenters + mesh.fn * normal_display_length)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016840…

## Defining neighbourhoods for pointclouds
There are 2 methods implemented for neighbourhood definitions. Creating a KNN graph on the pointcloud and creating a robust laplacian. The second method is a lot slower then the first method, but the second method probably represents a better neighbourhood definition.

In [4]:
knn_pointcloud_graph = pointcloud.toGraph(0)
robust_pointcloud_graph = pointcloud.toGraph(1)
knn_edges = knn_pointcloud_graph.edge_index
robust_edges = robust_pointcloud_graph.edge_index

In the image below, the black edges are created by the knn neighbourhood for k=12 and the blue edges are created by the robust laplacian method. 3 visualizations are shown to illustrate the difference of the assignment of the edges.

In [5]:
distance = (igl_bounding_box_diagonal(pointcloud.v)**2/3)**0.5
num_v = pointcloud.v.shape[0]
vertices = np.r_[pointcloud.v, pointcloud.v + distance*np.array([1, 0, 0]), pointcloud.v + distance*np.array([2, 0, 0])]
plot = mp_plot(vertices, shading={"point_color": "green", "point_size": 0.00000001})
plot.add_lines(vertices[knn_edges[0]], vertices[knn_edges[1]], shading={"line_color": "black", "line_width": 0.00001})
plot.add_lines(vertices[robust_edges[0]+num_v], vertices[robust_edges[1]+num_v], shading={"line_color": "blue", "line_width": 0.00001})
plot.add_lines(vertices[knn_edges[0]+2*num_v], vertices[knn_edges[1]+2*num_v], shading={"line_color": "black", "line_width": 0.00001})
_ = plot.add_lines(vertices[robust_edges[0]+2*num_v], vertices[robust_edges[1]+2*num_v], shading={"line_color": "blue", "line_width": 0.00001})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.1276394…

The same can be done for the mesh variant. In this case, faces are graph nodes and if 2 faces are connected to each other, there exists an edge between the 2 nodes representing the faces.

In [6]:
mesh_graph = mesh.toGraph()
# plot = mp_plot(mesh.v, mesh.f, shading={"wireframe": "true"})
plot = mp_plot(mesh_graph.pos.numpy(), shading={"point_size": 0.000000001})
vertices = mesh_graph.pos
edges = mesh_graph.edge_index
_ = plot.add_lines(vertices[edges[0]], vertices[edges[1]])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016837…

## Create artificial noise (Store Ground Truth)
Below you can generate noise for the pointcloud and for the mesh. The 3 variables at the top determine what kind of noise it will be.

In [7]:
EXAMPLE_NOISE_LEVEL = 0.1
EXAMPLE_NOISE_DIRECTION = 0
EXAMPLE_NOISE_TYPE = 0

pointcloud_noise = Noise(pointcloud=pointcloud)
mesh_noise = Noise(pointcloud=mesh)
pointcloud_noise.generateNoise(EXAMPLE_NOISE_LEVEL, noise_direction=EXAMPLE_NOISE_DIRECTION, noise_type=EXAMPLE_NOISE_TYPE)
mesh_noise.generateNoise(EXAMPLE_NOISE_LEVEL, noise_direction=EXAMPLE_NOISE_DIRECTION, noise_type=EXAMPLE_NOISE_TYPE)

_v = pointcloud.gt
_pos = pointcloud.v
_vn = pointcloud.vn
x1 = np.vstack((_v, _pos))
c1 = np.zeros((x1.shape[0],))
c1[_v.shape[0]:] = 1
_v = mesh.gt
_pos = mesh.v
_vn = mesh.vn
x2 = np.vstack((_v, _pos))
f2 = np.vstack((mesh.f, mesh.f + len(mesh.v)))
c2 = np.zeros((x2.shape[0],))
c2[_v.shape[0]:] = 1

plot1 = mp_plot(x1, c=c1, shading={"point_size": 0.003})
normals = _vn/500
_ = plot1.add_lines(_v, _v+normals, shading={"line_width": 0.003})
plot2 = mp_plot(x2, f2, c=c2, shading={"point_size": 0.003})
normals = _vn/500
_ = plot2.add_lines(_v, _v+normals, shading={"line_width": 0.003})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016836…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016881…

Below, we will show that saving and loading a noise object will result in the same noise object.

In [8]:
save_filename = pointcloud_noise.saveNoise()
_v1 = deepcopy(pointcloud_noise.object.v)
pointcloud_noise.resetNoise()

# Pointcloud has actually been reset to ground truth
print(np.allclose(pointcloud_noise.object.v, pointcloud_noise.object.gt))

pointcloud_noise.loadNoise(save_filename)
_v2 = deepcopy(pointcloud_noise.object.v)

# Pointcloud after loading is the same as when saving.
print(np.allclose(_v1, _v2))

True
True


## Patch Selection
In this section, we will try to prove with visualizations that we can select patches based on a distance metric for pointclouds and meshes.

In [9]:
patchgenerator_pointcloud = PatchGenerator(pointcloud)
patchgenerator_mesh = PatchGenerator(mesh)

In [14]:
nodes = patchgenerator_pointcloud.toPatches()

Started collecting patches: Collecting Tworings
Collected Tworings. Starting to collect radii
Collected radii, starting the ball query
Collected bearby vertices. Starting to collect nodes from vertices.


In [15]:
c = [nodes[5], nodes[100], nodes[800]]
c.reverse()
visualize_coloring(patchgenerator_pointcloud.object.g.pos.numpy(), c)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.016836…

dtype('O')